In [ ]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import pandas as pd
from wulpus.helper import zip_to_dataframe, find_latest_measurement_zip
from wulpus.plot_helpers import flatten_df_measurements, imshow_with_time
import os
import glob
import wulpus as wulpus_pkg
import inspect

In [ ]:
path = find_latest_measurement_zip()
# path = ".\\wulpus\\measurements\\wulpus-2025-09-05_11-01-36.zip"
SAMPLE_CROP = 100 #  no crop e.g. 99999

In [ ]:
df, config = zip_to_dataframe(path)
print(df.info())
df.head()

In [ ]:
data_sel = np.stack(df['measurement'].to_numpy())
channel_id = 0

plt.figure(figsize=(10, 5))
plot_data = plt.plot(data_sel[0], linestyle='-', marker='o', linewidth=1, markersize=2)
plt.ylim(-2500, 2500)
plt.title(f'(Channel {channel_id})')
plt.xlabel('Samples')
plt.ylabel('ADC digital code')
plt.grid()
plt.show()

def visualize(frame):
    plot_data[0].set_ydata(data_sel[frame])
    plt.title(f'Acquisition {frame}/{data_sel.shape[0]-1} (Channel {channel_id})')
    plt.draw()

widgets.interact(visualize, frame=widgets.IntSlider(min=0, max=data_sel.shape[0]-1, step=1, value=0))

In [ ]:
# Grouped heatmaps by tx_rx_id with shared colorbar and shared x-label
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

groups = list(df.groupby('tx_rx_id'))
n_groups = len(groups)
if n_groups == 0:
    raise ValueError('No groups found for tx_rx_id')

# First pass: compute global vmin/vmax across all groups (after flattening and crop)
global_min, global_max = None, None
flattened_cache = []
for txrx, group_df in groups:
    flat_df, meas_cols = flatten_df_measurements(group_df, sample_crop=SAMPLE_CROP)
    values = flat_df[meas_cols].to_numpy(dtype=float)  # (n_acq, n_samples)
    plot_data = values.T  # (n_samples, n_acq)
    vmin = np.nanmin(plot_data) if plot_data.size else None
    vmax = np.nanmax(plot_data) if plot_data.size else None
    flattened_cache.append((txrx, group_df, flat_df, meas_cols, plot_data))
    if vmin is not None:
        global_min = vmin if global_min is None else min(global_min, vmin)
    if vmax is not None:
        global_max = vmax if global_max is None else max(global_max, vmax)

# Build a common norm
if global_min is None or global_max is None:
    global_min, global_max = -1, 1
norm = colors.SymLogNorm(linthresh=10.0, linscale=0.03, vmin=global_min, vmax=global_max)

# Prepare axes: one row per group, share x among axes for common x-label
fig, axs = plt.subplots(nrows=n_groups, ncols=1, figsize=(10, 2.2 * n_groups), sharex=True)
if n_groups == 1:
    axs = [axs]

# Second pass: draw each subplot (no per-axes colorbar)
ims = []
for i, (ax, cached) in enumerate(zip(axs, flattened_cache)):
    txrx, group_df, flat_df, meas_cols, plot_data = cached
    # Draw image without adding colorbar (shared colorbar below)
    im = imshow_with_time(ax, plot_data, index=group_df.index, cmap='viridis', interpolation='hamming', norm=norm, num_ticks=10, add_colorbar=False)
    ims.append(im)

    # Title with config details if available
    try:
        cfg = config.tx_rx_config[txrx]
        title_extra = f"  TX {cfg.tx_channels} -> RX {cfg.rx_channels}"
    except Exception:
        title_extra = ''
    ax.set_title(f'tx_rx_id: {txrx} (n={len(group_df)})' + title_extra)

    # Y ticks
    n_samples = plot_data.shape[0]
    sample_ticks = np.linspace(0, n_samples - 1, min(6, max(1, n_samples)), dtype=int)
    ax.set_yticks(sample_ticks)
    ax.set_yticklabels([str(t) for t in sample_ticks])

    # Hide x tick labels for all but the bottom axis (since sharex=True)
    if i < n_groups - 1:
        ax.tick_params(axis='x', labelbottom=False)
    ax.set_ylabel('Sample index')

# Shared colorbar across all subplots at bottom, horizontal
# cbar = fig.colorbar(ims[0],
#                     ax=axs,
#                     location='bottom',
#                     orientation='horizontal',
#                     fraction=0.06,
#                     pad=0.12,
#                     shrink=0.7,
#                     anchor=((0.5, 1.0)))
# cbar.set_label('ADC digital code')


axins = inset_axes(ax,
                    width="100%",  
                    height="5%",
                    loc='lower center',
                    borderpad=-7
                   )
cbar = fig.colorbar(im, cax=axins, orientation="horizontal")
cbar.set_label('ADC digital code')


# Shared x-label at the figure level
fig.supxlabel('Time (acquisition)')

plt.tight_layout()
plt.show()

In [ ]:
# Plot with real time on x-axis from DataFrame index (helper-based)
fig, ax = plt.subplots(figsize=(12, 6))
border = 1000

# Select a single RX channel (0) and flatten to measurement columns
df_channel_sel = df[df['rx'] == 0]
flattened_df, meas_cols = flatten_df_measurements(df_channel_sel, sample_crop=SAMPLE_CROP)

# Build (n_acq, n_samples) and transpose to (n_samples, n_acq) for imshow
values = flattened_df[meas_cols].to_numpy(dtype=float)
plot_data = values.T

# Choose a robust norm (symmetric range around 0 for visibility)
norm = colors.SymLogNorm(linthresh=10, linscale=0.03, vmin=-border, vmax=border)

# Plot using helper (adds colorbar and local-time xticks)
imshow_with_time(ax, plot_data, index=df_channel_sel.index, cmap='viridis', interpolation='hamming', norm=norm, num_ticks=10)

ax.set_title('Measurement evolution over time (RX 0)')
ax.set_xlabel('Time')
ax.set_ylabel('Sample index within measurement (distance of reflection)')
ax.grid(False)

plt.tight_layout()
plt.show()

In [ ]:
# Find the most recent measurement file
measurement_dir = os.path.join(os.path.dirname(
    inspect.getfile(wulpus_pkg)), 'measurements')
if not os.path.exists(measurement_dir):
    raise FileNotFoundError(
        f"Measurements directory not found: {measurement_dir}")

zip_files = glob.glob(os.path.join(measurement_dir, '*.zip'))
if not zip_files:
    raise FileNotFoundError(f"No zip files found in {measurement_dir}")

# Use the most recent file
path = max(zip_files, key=os.path.getctime)

try:
    df, config = zip_to_dataframe(path)
except Exception as e:
    raise RuntimeError(f"Failed to load data from {path}: {e}")

# Check if measurement column exists and has data
if 'measurement' not in df.columns or df.empty:
    raise ValueError("Something's off with your file")

# Handle case where measurements might be empty or have different formats
measurements = df['measurement'].dropna()

# Create interactive 3D plot
fig = plt.figure(figsize=(14, 10))
border = 1000

# Compute max length of measurements to handle varying lengths
max_len = max(len(m) if hasattr(m, '__len__') else 1 for m in measurements)

df_channel_sel = df[df['rx'] == 0]
padded = np.array([np.array(m, dtype=float)
                  for m in df_channel_sel['measurement']])
padded = padded[:, :100]

# Create result DataFrame with proper indexing
measurement_df = pd.DataFrame(padded, index=df_channel_sel.index)
result = pd.concat(
    [df.drop(columns=['measurement']).loc[df_channel_sel.index], measurement_df], axis=1)

# Create 3D surface plot
ax = fig.add_subplot(111, projection='3d')

# Create coordinate grids for 3D plotting
X = np.arange(padded.shape[0])  # Time (measurement index)
Y = np.arange(padded.shape[1])  # Sample index within measurement
X, Y = np.meshgrid(X, Y)

# Transpose padded data to match meshgrid orientation
Z = padded.T

# Create the 3D surface plot
surf = ax.plot_surface(X, Y, Z, 
                      cmap='viridis', 
                      alpha=0.8,
                      linewidth=0,
                      antialiased=True,
                      vmin=-border, 
                      vmax=border)

# Add a color bar
cbar = fig.colorbar(surf, ax=ax, shrink=0.5, aspect=20)
cbar.set_label('ADC digital code', rotation=270, labelpad=15)

# Set labels and title
ax.set_xlabel('Time')
ax.set_ylabel('Sample index within measurement (distance of reflection)')
ax.set_zlabel('ADC digital code')
ax.set_title('3D Interactive Measurement Evolution Over Time')

# Set viewing angle for better initial perspective
ax.view_init(elev=30, azim=45)

# Set z-axis limits for better visualization
ax.set_zlim(-border, border)

# Enable interactive rotation and zooming
plt.tight_layout()
plt.show()
